# Q-Learning: Continuous Example with CartPole

This notebook implements the Q-Learning algorithm for the [CartPole](https://gym.openai.com/envs/CartPole-v1/) game.
See `../ReinforcementLearning_Guide.md` for theory and intuition.

According to the OpenAI environment page of CartPole: "A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center."

Overview of sections:

1. Basic Setup of CartPole